In [1]:
import pandas as pd
import numpy as np
import S2_0_Loading_Data
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, TimeDistributed
from keras.callbacks import EarlyStopping
import keras
import S4_0_Helper_Functions

In [2]:
train_x, test_x, train_y, test_y = S2_0_Loading_Data.load_data(image_size=(100, 100))
train_x.shape

(3257, 100, 100, 3)

In [3]:
inp = keras.Input(shape=(100, 100, 3))
x = TimeDistributed(LSTM(3, return_sequences=True))(inp)
model_LSTM = keras.Model(inp, x)
model = Sequential()
model.add(model_LSTM)
model.add(Dropout(0.1)),
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(3, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 100, 100, 3)       84        
                                                                 
 dropout (Dropout)           (None, 100, 100, 3)       0         
                                                                 
 flatten (Flatten)           (None, 30000)             0         
                                                                 
 dense (Dense)               (None, 256)               7680256   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 7,681,111
Trainable params: 7,681,111
Non-

In [4]:
# compile model
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='CategoricalCrossentropy',
              metrics=METRICS)

In [5]:
callback = EarlyStopping(monitor='val_prc', mode='max',min_delta = 0.01, verbose=1, patience=10,restore_best_weights=True)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 1, # change to 100 to get superior
    verbose = 1,
    callbacks = [callback]
)

102/102 [==============================] - 17s 146ms/step - loss: 0.9008 - tp: 2283.0000 - fp: 561.0000 - tn: 5953.0000 - fn: 974.0000 - accuracy: 0.7670 - precision: 0.8027 - recall: 0.7010 - auc: 0.9107 - prc: 0.8464 - val_loss: 0.6231 - val_tp: 569.0000 - val_fp: 136.0000 - val_tn: 1494.0000 - val_fn: 246.0000 - val_accuracy: 0.7607 - val_precision: 0.8071 - val_recall: 0.6982 - val_auc: 0.9181 - val_prc: 0.8575


In [6]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.7607361963190185
recall: 0.7607361963190185
precision: 0.7607361963190185
